<a href="https://colab.research.google.com/github/jayantc14/Yelp_Data_Analysis/blob/main/Labels_from_BERT_yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 41 kB 171 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=cfc350ec5f621c1dc60bf6330f1c23acf4f2a99e07bfbbea93f93361096e1b7b
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=620c28ce372e0d497f1cb5a8f944103c2601d064a93ab98038bed7e91a90256b
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=96a1866ca12ebe02df4c37e3744038f8863c9fef3cd23d62b7e3981a118142ac
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 25.9 MB/s 


In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("/content/drive/MyDrive/yelp_dataset/final_data_yelp2.csv")


In [5]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'business_id', 'city', 'state_x',
       'user_id', 'review_stars', 'useful_x', 'date', 'useful_y',
       'fans', 'Smogscore', 'review_word_count', 'Polarity',
       'Analysis_TextBlob', 'Vader Sentiment', 'Vader Analysis', 'fips',
       'cases', 'deaths'],axis = 1,inplace = True)

In [6]:
movie_reviews = df
movie_reviews.shape

(197949, 2)

In [7]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [8]:
import re

In [9]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [10]:
reviews = []
sentences = list(movie_reviews['text'])
for sen in sentences:
    reviews.append(preprocess_text(sen))

In [11]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [12]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [13]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

[2123, 2102, 2022, 2061, 8689, 2389]

In [14]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [15]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [16]:
import numpy as np

In [17]:
y = df['label']

y = np.array(list(map(lambda x: 1 if x== 1.0 else 0, y)))

In [18]:
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]

In [19]:
#import random


In [20]:
#random.shuffle(reviews_with_len)

In [21]:
#reviews_with_len.sort(key=lambda x: x[2])

In [22]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [23]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [24]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [25]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 488), dtype=int32, numpy=
 array([[ 2748,  2572,  5470, ...,     0,     0,     0],
        [10047,  2025, 11957, ...,     0,     0,     0],
        [10392, 14469,  3059, ...,     0,     0,     0],
        ...,
        [ 2096,  2017,  2064, ...,     0,     0,     0],
        [ 2064,  2102,  2360, ...,     0,     0,     0],
        [ 2398,  2091,  2023, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1], dtype=int32)>)

In [ ]:
import math

In [ ]:
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [26]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [27]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 1

In [28]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [29]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [30]:
text_model.fit(batched_dataset, epochs=NB_EPOCHS)

6186/6186 [==============================] - 2206s 356ms/step - loss: 0.1326 - accuracy: 0.9454


In [31]:
y_pred = text_model.predict(batched_dataset)

In [32]:
y_pred

array([[0.4158437 ],
       [0.9832674 ],
       [0.99840057],
       ...,
       [0.9999175 ],
       [0.76743865],
       [0.80327976]], dtype=float32)

In [33]:
y_pred = (y_pred >= 0.5).astype("int")

In [34]:
y_pred

array([[0],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
train_data

<SkipDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [35]:
datas = pd.DataFrame(y_pred)

In [36]:
datas

,0
0,0
1,1
2,1
3,1
4,1
...,...
197944,1
197945,1
197946,1
197947,1


In [37]:
df

,text,label
0,Yes I am a fan of the food Ive tried more ...,1.0
1,Im not a bubble tea connoisseur but Ive had it...,1.0
2,Fantastic authentic Italian food the pinsa ha...,1.0
3,Good food very authentic The owner is pretty f...,1.0
4,The service is outstanding and the pasta is go...,1.0
...,...,...
197944,I have to admit Shin Hakata Ramen was truly im...,1.0
197945,The BEST Rueben in Boston Hands down And I hav...,1.0
197946,Mike Pattys is such a cool neighborhood gem I...,1.0
197947,Store is a little hard to find since it is hid...,1.0


In [38]:
datas.to_csv('/content/drive/MyDrive/yelp_dataset/dummy_train_2.csv')

In [43]:
datas[0] = datas[0].astype(float)

In [45]:
datas

,0
0,0.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
197944,1.0
197945,1.0
197946,1.0
197947,1.0


In [46]:
sum=0
for i in range(len(df)):
    if (df.loc[i,'label'] == datas.loc[i,0]):
        sum = sum+1


In [47]:
sum

189399

In [48]:
datas['bert_labels'] = datas[0]

In [49]:
datas.drop([0],axis = 1,inplace = True)

In [50]:
datas

,bert_labels
0,0.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
197944,1.0
197945,1.0
197946,1.0
197947,1.0


In [51]:
datas.to_csv('/content/drive/MyDrive/yelp_dataset/bert_labels.csv')

In [ ]:
datas

,0
0,1
1,1
2,1
3,1
4,1
...,...
178168,1
178169,1
178170,1
178171,1


In [39]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197949 entries, 0 to 197948
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   0       197949 non-null  int64
dtypes: int64(1)
memory usage: 1.5 MB


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197949 entries, 0 to 197948
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   text    197949 non-null  object 
 1   label   197949 non-null  float64
dtypes: float64(1), object(1)
memory usage: 3.0+ MB


In [ ]:
train_data

<SkipDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [ ]:
trn = pd.DataFrame(train_data)

In [ ]:
tst = pd.DataFrame(test_data)

In [ ]:
test_data

<TakeDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [ ]:
next(iter(train_data))

(<tf.Tensor: shape=(32, 25), dtype=int32, numpy=
 array([[ 6429, 28496, 18249,  3672,  4495,  2293,  1996,  2572, 15599,
          3401,  1998, 25545,  3745,  3085,  2833,  2001,  9805, 18879,
          2978,  3976,  2100,  2021,  4276,  1996,  3325],
        [ 6429,  4777,  1998,  1996, 20949,  7635,  2024,  6429,  2065,
          2973,  2105,  2182,  2023,  2052,  2022,  2026,  3180,  3962,
          1996,  7597,  2024,  3565,  9608,  2004,  2092],
        [ 5024,  2822, 13258,  2012,  4189,  3976,  1996,  7224,  3727,
          6655,  2000,  2022,  9059,  2021,  2023,  2003,  2202,  5833,
         17070,  2007,  5024,  4841,  2197,  2822, 12846],
        [ 2293,  1996, 21209,  2015,  4658, 25545,  2015,  1998, 15184,
          8974,  3811,  6749,  2000,  3071,  8827,  2049,  2524,  2000,
          2424,  5581,  2105,  2045, 19169,  2045,  2612],
        [ 2123,  2102,  2344,  1996,  7975, 13017, 21475,  2049,  7852,
          2098, 24575, 17798,  1996,  2326,  2001,  3835,  1996,  8

In [ ]:
next(iter(test_data))

(<tf.Tensor: shape=(32, 0), dtype=int32, numpy=array([], shape=(32, 0), dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>)

In [ ]:
y_pred2 = text_model.predict(test_data)

InvalidArgumentError: ignored

In [ ]:
#Prediction is in probability of news being real, so converting into classes
# Class 0 (Fake) if predicted prob < 0.5, else class 1 (Real)
y_pred

In [ ]:
results = text_model.evaluate(test_data)
print(results)

InvalidArgumentError: ignored